In [1]:
# 1. Import bibliotek
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from utils.dataset_tools import DatasetTools

2025-08-18 17:56:02.059354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755532562.075310   12717 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755532562.080082   12717 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755532562.092837   12717 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755532562.092857   12717 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755532562.092858   12717 computation_placer.cc:177] computation placer alr

In [2]:
# 2. Załadowanie danych, przygotowanie
df = pd.read_csv('../dataset/full_dataset.csv')
df.head()

,sample_id,frame,label,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,landmark_2_y,landmark_2_z,landmark_3_x,...,landmark_18_z,landmark_19_x,landmark_19_y,landmark_19_z,landmark_20_x,landmark_20_y,landmark_20_z,landmark_21_x,landmark_21_y,landmark_21_z
0,1,0,A,0.345417,0.648431,-1.125724e-07,0.363545,0.620765,-0.010274,0.367171,...,-0.003865,0.321722,0.551512,-0.008457,0.326903,0.565844,-0.005724,0.329682,0.577548,-0.001789
1,1,1,A,0.366266,0.597102,-9.622672e-08,0.391580,0.578109,-0.012907,0.406190,...,0.000685,0.356418,0.497935,-0.004451,0.361665,0.515749,-0.001988,0.363353,0.523833,0.002337
2,1,2,A,0.400966,0.572690,-1.089629e-07,0.424555,0.552354,-0.004017,0.439723,...,-0.004296,0.383425,0.481909,-0.009304,0.389207,0.502081,-0.005430,0.391897,0.513775,-0.000924
3,1,3,A,0.418257,0.573744,-1.190043e-07,0.441344,0.563031,-0.007786,0.460869,...,-0.002903,0.402536,0.489501,-0.009669,0.406139,0.510639,-0.006567,0.409394,0.523706,-0.001918
4,1,4,A,0.431913,0.582072,-1.241054e-07,0.453692,0.577044,-0.009167,0.474907,...,-0.004793,0.417201,0.501486,-0.012369,0.419653,0.524298,-0.008715,0.422120,0.535854,-0.003439


In [3]:
# 3. Podział na 30 klatkowe sekwencje
X_all, y_all = DatasetTools.divide_into_sequences(df)

In [4]:
# 4. Normalizacja względem nadgarstka
X_all = np.array([DatasetTools.normalize_wrist(seq) for seq in X_all])

In [5]:
# 5. Augmentacja danych
augment_config = {
    'flip': True,
    'gaussian_noise': True,
    'speckle_noise': False,
    'salt_pepper': False
}

X_all_aug = np.array([DatasetTools.augment(tf.constant(seq, dtype=tf.float64), augment_config).numpy() for seq in X_all])

I0000 00:00:1755532568.907295   12717 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1733 MB memory:  -> device: 0, name: NVIDIA GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1


In [6]:
# 6. Podział na zbiór testowy i treningowy
from sklearn.preprocessing import LabelEncoder

# Kodowanie labeli
le = LabelEncoder()
y_all_encoded = le.fit_transform(y_all)

# Podział na zbiór treningowy i walidacyjny
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_all, y_all_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_all_encoded
)

In [7]:
# 7. Konwersja do TensorFlow Dataset
BATCH_SIZE = 32

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [8]:
# 8. Definicja prostego LSTM
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(DatasetTools.SEQUENCE_SIZE, DatasetTools.FEATURE_SIZE)),
    layers.Masking(mask_value=0.0),
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(64),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(y_all)), activation='softmax')
])


In [9]:
# 9. Kompilacja i trening
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,            # liczba epok bez poprawy, po której trening się zatrzyma
    restore_best_weights=True
)

# Trening modelu z early stopping
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stop]
)


Epoch 1/100


I0000 00:00:1755532611.434635   12769 cuda_dnn.cc:529] Loaded cuDNN version 90300


560/560 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.3413 - loss: 2.0589 - val_accuracy: 0.7258 - val_loss: 0.7672
Epoch 2/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 19s 35ms/step - accuracy: 0.7571 - loss: 0.6404 - val_accuracy: 0.8688 - val_loss: 0.3466
Epoch 3/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.8218 - loss: 0.4847 - val_accuracy: 0.8411 - val_loss: 0.3841
Epoch 4/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.8802 - loss: 0.3192 - val_accuracy: 0.9097 - val_loss: 0.2555
Epoch 5/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.9115 - loss: 0.2542 - val_accuracy: 0.9120 - val_loss: 0.2380
Epoch 6/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.9187 - loss: 0.2323 - val_accuracy: 0.9292 - val_loss: 0.1906
Epoch 7/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.9328 - loss: 0.1943 - val_accuracy: 0.9267 - val_loss: 0.2042
Epoch 8/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.9243 - loss: 0.2220 - val_

In [13]:
# 10. Ocena skuteczności modelu
val_preds = model.predict(val_dataset)
val_preds_classes = np.argmax(val_preds, axis=1)

# Pobranie prawdziwych etykiet z datasetu
y_val_true = np.concatenate([y for x, y in val_dataset], axis=0)

print("Confusion Matrix:")
print(confusion_matrix(y_val_true, val_preds_classes))

print("\nClassification Report:")
print(classification_report(y_val_true, val_preds_classes))

140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Confusion Matrix:
[[119   0   0 ...   0   0   0]
 [  6 118   0 ...   0   0   0]
 [  0   0 124 ...   0   0   0]
 ...
 [  0   0   0 ... 119   7   0]
 [  0   0   0 ...  10 112   3]
 [  1   0   0 ...   3   0 120]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       119
           1       1.00      0.95      0.98       124
           2       1.00      1.00      1.00       124
           3       1.00      0.97      0.98       124
           4       1.00      0.99      1.00       124
           5       1.00      0.98      0.99       124
           6       0.99      1.00      1.00       125
           7       0.98      0.99      0.98       124
           8       0.96      0.98      0.97       125
           9       1.00      0.96      0.98       124
          10       1.00      0.98      0.99       125
          11       0.99      1.00      1.00       124
          12       0.9

2025-08-18 18:19:26.224549: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
import os
from datetime import datetime
import json

# Ustal dzisiejszą datę
today_str = datetime.now().strftime("%Y-%m-%d")

# Ścieżka zapisu modeli
save_dir = f"../model/{today_str}"
os.makedirs(save_dir, exist_ok=True)

# Ścieżki plików
model_path = os.path.join(save_dir, "lstm.keras")
labels_path = os.path.join(save_dir, "labels.txt")

# Zapis modeli
model.save(model_path)

with open('../model/2025-07-14/labels.txt', 'w') as f:
    for label in le.classes_:
        f.write(label + '\n')

print(f"Modele i etykiety zapisano w folderze: {save_dir}")


Modele i etykiety zapisano w folderze: ../model/2025-08-18
